In [1]:
import os
import tensorflow as tf
import numpy as np
import DEN
from tensorflow.examples.tutorials.mnist import input_data
import sys

np.random.seed(1004)

flags = tf.app.flags
flags.DEFINE_integer("max_iter", 2000, "Epoch to train")
flags.DEFINE_float("lr", 0.001, "Learing rate(init) for train")
flags.DEFINE_integer("batch_size", 256, "The size of batch for 1 iteration")
flags.DEFINE_string("checkpoint_dir", "checkpoints", "Directory path to save the checkpoints")
flags.DEFINE_integer("dims0", 43, "Dimensions about input layer")
flags.DEFINE_integer("dims1", 60, "Dimensions about 1st layer")
flags.DEFINE_integer("dims2", 30, "Dimensions about 2nd layer")
flags.DEFINE_integer("dims3", 10, "Dimensions about output layer")
flags.DEFINE_integer("n_classes", 10, 'The number of classes at each task')
flags.DEFINE_float("l1_lambda", 0.00001, "Sparsity for L1")
flags.DEFINE_float("l2_lambda", 0.0001, "L2 lambda")
flags.DEFINE_float("gl_lambda", 0.001, "Group Lasso lambda")
flags.DEFINE_float("regular_lambda", 0.05, "regularization lambda")
flags.DEFINE_integer("ex_k", 150, "The number of units increased in the expansion processing")
flags.DEFINE_float('loss_thr', 0.01, "Threshold of dynamic expansion")
flags.DEFINE_float('spl_thr', 0.05, "Threshold of split and duplication")
FLAGS = flags.FLAGS

In [2]:
import pickle
unsw_tasks = pickle.load(open('unsw_tasks.pkl', "rb"))
n_tasks = len(unsw_tasks)

In [3]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

import pdb

model = DEN.DEN(FLAGS)
params = dict()
avg_perf = []

for m in range(0,n_tasks): #(FLAGS.n_classes):
    tasked_input = unsw_tasks[m][0]
    tasked_output = unsw_tasks[m][1]
    print (tasked_input.shape,tasked_output.shape)
    data = (unsw_tasks[m][0],unsw_tasks[m][1],unsw_tasks[m][2],unsw_tasks[m][3],unsw_tasks[m][4],unsw_tasks[m][5])
    model.sess = tf.Session()
    print("\n\n\tTASK %d TRAINING\n"%(m+1))

    model.T = model.T+1
    model.task_indices.append(m+1)
    model.load_params(params, time = 1)
    perf, sparsity, expansion = model.add_task(m+1, data)
    
    params = model.get_params()
    #pdb.set_trace()
    #analyze_params(params)
    model.destroy_graph()
    model.sess.close()

    model.sess= tf.Session()
    print('\n OVERALL EVALUATION')
    model.load_params(params)
    temp_perfs = []
    for j in range(m+1):
        temp_perf = model.predict_perform(j+1, tasked_input, tasked_output)
        temp_perfs.append(temp_perf)
    avg_perf.append( sum(temp_perfs) / float(m+1) )
    print("   [*] avg_perf: %.4f"%avg_perf[m])
    model.destroy_graph()
    model.sess.close()



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(1260, 43) (1260, 10)


	TASK 1 TRAINING



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



 [*] iter: 100, val loss: 0.2998, val perf: 0.5013
 [*] iter: 200, val loss: 0.1887, val perf: 0.4868
 [*] iter: 300, val loss: 0.1745, val perf: 0.4681
 [*] iter: 400, val loss: 0.1713, val perf: 0.4624
 [*] iter: 500, val loss: 0.1709, val perf: 0.4600
 [*] iter: 600, val loss: 0.1714, val perf: 0.4591
 [*] iter: 700, val loss: 0.1723, val perf: 0.4585
 [*] iter: 800, val loss: 0.1734, val perf: 0.4578
 [*] iter: 900, val loss: 0.1746, val perf: 0.4571
 [*] iter: 1000, val loss: 0.1758, val perf: 0.4565
 [*] iter: 1100, val loss: 0.1770, val perf: 0.4560
 [*] iter: 1200, val loss: 0.1783, val perf: 0.4555
 [*] iter: 1300, val loss: 0.1797, val perf: 0.4550
 [*] iter: 1400, val loss: 0.1810, val perf: 0.4544
 [*] iter: 1500, val

 [*] iter: 1800, val loss: 0.0455, val perf: 0.9478
 [*] iter: 1900, val loss: 0.0439, val perf: 0.9477
 [*] iter: 2000, val loss: 0.0453, val perf: 0.9448
   [*] split, loss: 0.0083, nn_perf: 0.9448(valid) repeated: 2000
   [*] Total expansions: [42, 82]
 [*] hidden 1, shape : [43, 210]
 [*] hidden 2, shape : [210, 258]
 [*] Task: 4, nn_test_loss: 0.0453, test_perf: 0.9448, sparsity(avg): 0.1808

 OVERALL EVALUATION
 [*] hidden 1, shape : [43, 60]
 [*] hidden 2, shape : [60, 30]
 [*] Evaluation, Task:1, test_acc: 0.5099
 [*] hidden 1, shape : [43, 149]
 [*] hidden 2, shape : [149, 96]
 [*] Evaluation, Task:2, test_acc: 0.5000
 [*] hidden 1, shape : [43, 168]
 [*] hidden 2, shape : [168, 176]
 [*] Evaluation, Task:3, test_acc: 0.5000
 [*] hidden 1, shape : [43, 210]
 [*] hidden 2, shape : [210, 258]
 [*] Evaluation, Task:4, test_acc: 0.9922
   [*] avg_perf: 0.6255
(422, 43) (422, 10)


	TASK 5 TRAINING

 [*] Selective retraining
 [*] Network expansion (training)
   [*] Expanding 2th hi